In [9]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [10]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(url)
results_page = BeautifulSoup(response.content, "html.parser")

In [11]:
table = results_page.find("table")
table_items = table.find_all("tr")
table_head = table_items[0].find_all("th")
table_rows = table_items[1:]

In [12]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
toronto_neighborhoods = pd.DataFrame(columns = column_names)

In [13]:
for row in table_rows:
    
    row = row.find_all("td")
    PostalCode = row[0].get_text()
    Borough = row[1].get_text()
    Neighborhood = row[2].get_text().strip("\n")
    
    if Neighborhood == "Not assigned" and Borough != "Not assigned":
        Neighborhood = Borough
    
    if Borough == "Not assigned":
        continue
    else:
        if PostalCode in toronto_neighborhoods["PostalCode"].tolist():
            index = toronto_neighborhoods["PostalCode"].tolist().index(PostalCode)
            toronto_neighborhoods.loc[[index], ["Neighborhood"]] = toronto_neighborhoods.loc[[index], ["Neighborhood"]] + ", " + Neighborhood 
        else:
            toronto_neighborhoods = toronto_neighborhoods.append({
                "PostalCode": PostalCode,
                "Borough": Borough,
                "Neighborhood": Neighborhood
            }, ignore_index = True)

In [14]:
toronto_neighborhoods

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z\n,Not assigned\n,Not assigned\n
176,M6Z\n,Not assigned\n,Not assigned\n
177,M7Z\n,Not assigned\n,Not assigned\n
178,M8Z\n,Etobicoke\n,"Mimico NW, The Queensway West, South of Bloor,..."


In [15]:
toronto_neighborhoods.shape

(180, 3)

In [16]:
lat_long_df = pd.read_csv("./Geospatial_Coordinates.csv")
toronto_neigh_with_coord = toronto_neighborhoods.join(lat_long_df.set_index(["Postal Code"]), on = "PostalCode")
toronto_neigh_with_coord

FileNotFoundError: [Errno 2] File b'./Geospatial_Coordinates.csv' does not exist: b'./Geospatial_Coordinates.csv'

In [17]:
import folium

ModuleNotFoundError: No module named 'folium'

In [18]:
map_Toronto = folium.Map(location=[43.715, -79.3832], zoom_start=10.5)

# add markers to map
for lat, lng, label in zip(toronto_neigh_with_coord['Latitude'], toronto_neigh_with_coord['Longitude'], toronto_neigh_with_coord['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7.5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

NameError: name 'folium' is not defined

In [19]:
CLIENT_ID = 'SQNBAVJBDG0POT5KFY2SVAOMWNUZB34ZVD4BR5XNFCNZXO3D' # your Foursquare ID
CLIENT_SECRET = '4KRJRO44XCYV0NLA5F5JROK2N5P3PQDODJEZYYFNCXQRJWRK' # your Foursquare Secret
VERSION = '20190601' # Foursquare API version

LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&time=any&day=any'.format(
    CLIENT_ID, CLIENT_SECRET, 43.657162, -79.378937, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=SQNBAVJBDG0POT5KFY2SVAOMWNUZB34ZVD4BR5XNFCNZXO3D&client_secret=4KRJRO44XCYV0NLA5F5JROK2N5P3PQDODJEZYYFNCXQRJWRK&ll=43.657162,-79.378937&v=20190601&radius=500&limit=100&time=any&day=any'

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&day=any&time=any'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
toronto_boroughs = toronto_neigh_with_coord[toronto_neigh_with_coord['Borough'].str.contains('Toronto')]
toronto_boroughs

NameError: name 'toronto_neigh_with_coord' is not defined